In [241]:
import pandas as pd
import pytesseract
from pytesseract import Output
import cv2
import json
from pdf2image import convert_from_path
import numpy as np
import time

In [38]:
#Converting PDF's to PNG

for file in os.listdir('Images'):
  if file.split('.')[-1]=='pdf':
    print(file)
    pdfFile = convert_from_path(f'images/{file}',dpi=300)
    
    for i in range(len(pdfFile)):
        # Save pages as images in the pdf
        pdfFile[i].save(f'PNGs/{file.split(".")[0]}_page{i}.png', 'PNG')

FGS8B39_2022.pdf
DKR8842_2022.pdf
EBB4B49_2022.pdf
ENU2816.pdf
DOC EEE4618.pdf
ESA2554.pdf
OAW9991 - CAVALO T1.pdf
ECT2365_2022.pdf


In [254]:
os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/Cellar/tesseract/5.2.0/share/tessdata'
multipliersDict = json.load(open('multipliersDictNew.json'))

In [255]:
whitelist = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZÇÁÉÍÓÚÃÕÂÊÎÔÛ-/'
blacklist = 'abcdefghijklmnopqrstuvwxyzçáéíóúãõâêîôû*&|º?[]\}{+=˚'

filename = 'DKR8842_2022_page0.png'
img = cv2.imread(f'PNGs/{filename}') 
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,img = cv2.threshold(img,230,255,cv2.THRESH_BINARY)
results = pytesseract.image_to_data(img, output_type=Output.DICT,lang='por',config=f'-c tessedit_char_blacklist={blacklist}')

img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)

In [256]:
#generating DF with the results from OCR detection and calculating new columns with the space between words

df = pd.DataFrame.from_dict(results)
df['right'] = df['left']+df['width']
df['space'] = np.nan
df['key'] = np.nan
df['spaceRelativeToHeight'] = np.nan

keyList = []
spaceRelativeToHeightList = []

for i in range(len(df)):
    keyList.append(str(df['page_num'][i])+str(df['block_num'][i])+str(df['par_num'][i])+str(df['line_num'][i]))
    try:
        diff = (df['left'][i]-df['right'][i-1])/df['height'][i-1]
        # if diff<0:
        #     diff=0
    except:
        pass
        # diff = np.nan

    spaceRelativeToHeightList.append(diff)
    
df['containsLower'] = df['text'].str.match(r'(?=.*[a-z])')
df['key'] = keyList
df['spaceRelativeToHeight'] = spaceRelativeToHeightList

In [257]:
#Concatenating nearby words into terms/phrases using new columns calculated

dfWords = df[(df['level']==5)&(df['containsLower']==False)&(df['text']!=' ')].reset_index().drop(columns=['index'])

phrases={}
phrase=''
currentKey = dfWords['key'][0]
left = dfWords['left'][0]
top = dfWords['top'][0]
width = dfWords['width'][0]
height = dfWords['height'][0]

width=0


for i in range(len(dfWords)):
    if (dfWords['spaceRelativeToHeight'][i]<=1) and (dfWords['key'][i]==currentKey):
        # print(dfWords['text'][i])
        phrase+=(dfWords['text'][i]+' ')
        width+=dfWords['width'][i]
    else:
        # print(phrase)
        phrases[phrase.strip()] = {'left':left, 'top':top, 'width':width, 'height':height}

        currentKey = dfWords['key'][i]
        left= dfWords['left'][i]
        top= dfWords['top'][i]
        height= dfWords['height'][i]
        width= dfWords['width'][i]
        phrase = (dfWords['text'][i]+' ')

In [267]:
#Plotting the rectangles over identified terms, using multiplier values 
#Cropping original images into smaller ones containing fields

whitelistCrop = '0123456789.'#ABCDEFGHIJKLMNOPQRSTUVWXYZÇÁÉÍÓÚÃÕÂÊÎÔÛ-/.*'
blacklistCrop = 'abcdefghijklmnopqrstuvwxyzçáéíóúãõâêîôû&º?[]}{+=˚|!$'

placaPlaceholder = file
img2=img

try:
    os.mkdir(f'Fields/{placaPlaceholder}')
except:
    pass

dictDataFrame = {}

for i in phrases.keys():
    if i in multipliersDict.keys():
        # print(i)
        x = int(phrases[i]['left'])-10
        y = int(phrases[i]['top'] + phrases[i]['height'])
        xw = int(phrases[i]['left'] + multipliersDict[i]['width']*(phrases[i]['width']))+10
        yh = int(phrases[i]['top'] + phrases[i]['height'] + multipliersDict[i]['height'])
        # print(y,yh,x,xw)
        # print(phrases[i]['top'])
        # print(phrases[i]['height'])
        # print(multipliersDict[i]['height'])
        crop_img = img[y:yh, x:xw]
        resultsCrop = pytesseract.image_to_data(crop_img, output_type=Output.DICT,lang='por',config=f'--psm 11 --oem 3 -c tessedit_char_blacklist={blacklistCrop}')
        
        fieldNameWords = i.split(' ')
        cropWords = resultsCrop['text']
        fieldAnswer = ' '.join(cropWords)
        
        if i == 'PLACA':
            placaReal = fieldAnswer.strip()
            print('Placa real', placaReal)
        dictDataFrame[str(i).replace('/','-')] = [fieldAnswer.strip()]

        print(i,'------>',fieldAnswer)
        cv2.imwrite(f"Fields/{placaPlaceholder}/{i.replace('/','-')}.png", crop_img)
        cv2.rectangle(img2, (x,y), (xw,yh), (0, 0, 255), 7)

# os.rename(f"Fields/{placaPlaceholder}",f"Fields/{placaReal}")
cv2.imwrite(f"Boxes/{placaReal}.png", img2)
# print(dictDataFrame)
dataFrameResults = pd.DataFrame.from_dict(dictDataFrame,orient='columns')
dataFrameResults.head()

CATEGORIA ------> 
CAPACIDADE ------> 
CÓDIGO RENAVAM ------>     01177650930
POTÊNCIA/CILINDRADA ------> 
PESO BRUTO TOTAL ------> 
Placa real DKR8842
PLACA ------>     DKR8842
EXERCÍCIO ------>     2022
MOTOR ------>     D13*8017908*C2*E
CMT ------>     60.0
EIXOS ------> 
LOTAÇÃO ------>     02P
ANO FABRICAÇÃO ------>     2018
ANO MODELO ------>     2019
CARROCERIA ------>     CABINE ESTENDIDA
NÚMERO DO CRV ------> 
NOME ------>     TRANSPORTADORA TRANSLECCHI LTDA
CPF / CNPJ ------>     60.746.518/0002-81
CÓDIGO DE SEGURANÇA DO CLA ------>     58239582459
CAT ------> 
LOCAL ------>     MOGI DA:    RUZE    P
DATA ------> 
MARCA / MODELO / VERSÃO ------> 
ESPÉCIE / TIPO ------>     TRACAO CAMINHAO TRATOR
PLACA ANTERIOR / UF ------>     *******/**
CHASSI ------>     9BVRG20C5KE862993
COR PREDOMINANTE ------>     BRANCA
COMBUSTÍVEL ------>     DIESEL


,CATEGORIA,CAPACIDADE,CÓDIGO RENAVAM,POTÊNCIA-CILINDRADA,PESO BRUTO TOTAL,PLACA,EXERCÍCIO,MOTOR,CMT,EIXOS,...,CÓDIGO DE SEGURANÇA DO CLA,CAT,LOCAL,DATA,MARCA - MODELO - VERSÃO,ESPÉCIE - TIPO,PLACA ANTERIOR - UF,CHASSI,COR PREDOMINANTE,COMBUSTÍVEL
0,,,01177650930,,,DKR8842,2022,D13*8017908*C2*E,60.0,,...,58239582459,,MOGI DA: RUZE P,,,TRACAO CAMINHAO TRATOR,*******/**,9BVRG20C5KE862993,BRANCA,DIESEL


In [266]:
dataFrameResults = pd.DataFrame(columns = ["COMBUSTÍVEL","COR PREDOMINANTE","CHASSI","PLACA ANTERIOR - UF","ESPÉCIE - TIPO","MARCA - MODELO - VERSÃO","DATA","LOCAL","CAT","CÓDIGO DE SEGURANÇA DO CLA","CPF - CNPJ","NOME","NÚMERO DO CRV","CARROCERIA","ANO MODELO","ANO FABRICAÇÃO","LOTAÇÃO","EIXOS","CMT","MOTOR","EXERCÍCIO","PLACA","PESO BRUTO TOTAL","POTÊNCIA-CILINDRADA","CÓDIGO RENAVAM","CAPACIDADE","CATEGORIA"])

dataFrameResults.head()

,COMBUSTÍVEL,COR PREDOMINANTE,CHASSI,PLACA ANTERIOR - UF,ESPÉCIE - TIPO,MARCA - MODELO - VERSÃO,DATA,LOCAL,CAT,CÓDIGO DE SEGURANÇA DO CLA,...,EIXOS,CMT,MOTOR,EXERCÍCIO,PLACA,PESO BRUTO TOTAL,POTÊNCIA-CILINDRADA,CÓDIGO RENAVAM,CAPACIDADE,CATEGORIA


In [221]:
img = cv2.imread(f'PNGs/{filename}') 

keywords = ['*']#list(multipliersDict.keys())

for i in range(0, len(results["text"])):
     if results['text'][i] in keywords:
        print(results['text'][i])
        # We can then extract the bounding box coordinates
        # of the text region from  the current result
        x = int(results["left"][i])
        y = int(results["top"][i])
        w = int(results["width"][i])
        h = int(results["height"][i])

        # We will also extract the OCR text itself along
        # with the confidence of the text localization
        text = results["text"][i]
        conf = int(results["conf"][i])
        page_num = results["page_num"][i]
        block_num = results["block_num"][i]
        par_num = results["par_num"][i]
        line_num = results["line_num"][i]
        word_num = results["word_num"][i]
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 0, 255), 10)

        if conf > 0:
            
            # We will display the confidence and text to
            # our terminal
            # print(f"Confidence: {conf}")
            print(f"Text: {text}")
            # print(f"page_num: {page_num}")
            # print(f"block_num: {block_num}")
            # print(f"par_num: {par_num}")
            # print(f"line_num: {line_num}")
            # print(f"word_num: {word_num}")
            # print("")

            
            # We then strip out non-ASCII text so we can
            # draw the text on the image We will be using
            # OpenCV, then draw a bounding box around the
            # text along with the text itself
            # text = "".join(text).strip()
            cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 0, 255), 2)
            # cv2.putText(img,
            #             text,
            #             (x, y - 10),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1.2, (0, 255, 255), 3)
	
cv2.imwrite(f"Boxes/{filename}", img)

True